In [1]:
import datetime
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    start="2020-01-01",
    end="2021-01-01",
    tickers="NDX",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [3]:
yfdata = response.copy().dropna()
yfdata = yfdata[yfdata.Volume != 0]
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,8802.22,8873.63,8786.90,8872.22,8872.22,2848370000
2020-01-03,8755.17,8843.65,8755.17,8793.90,8793.90,2567400000
2020-01-06,8713.89,8849.98,8713.89,8848.52,8848.52,2788120000
2020-01-07,8857.14,8872.47,8821.68,8846.45,8846.45,2352850000
2020-01-08,8845.45,8953.55,8834.94,8912.37,8912.37,2464090000
...,...,...,...,...,...,...
2020-12-24,12668.20,12732.55,12665.80,12711.01,12711.01,3305950000
2020-12-28,12813.97,12861.91,12747.05,12838.86,12838.86,5076340000
2020-12-29,12909.86,12925.53,12816.59,12843.49,12843.49,4680780000


In [21]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


def ATR(arr: pd.DataFrame) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["atr"]

In [50]:
class My_Strategy(Strategy):
    atr_line = 200

    take_profit = 4
    stop_loss = 6

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.atr = self.I(ATR, self.data.df)
        self.rsi = self.I(RSI, self.data.df, 7)

    def not_trade_with_atr(self):
        return self.atr[-1] < self.atr_line

    def is_over_sell(self):
        return self.rsi[-1] < 30

    def is_over_buy(self):
        return self.rsi[-1] > 70

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def next(self):

        # 計算できていない場合トレードしない
        if len(self.data.index) < 21:  # MACD
            return

        # Volatilityを判断して、変動幅が大きいタイミングでしかトレードしない
        if abs(1 - (self.atr[-1] / self.atr[-14])) * 100 < 5:
            print(abs(1 - (self.atr[-1] / self.atr[-14])) * 100)
            return

        # MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            if self.position.is_short:
                self.position.close()

            if self.position.is_long or self.is_over_buy():
                return

            self.buy(
                sl=self.data.Close[-1] - self.atr[-1] * 2,
                tp=self.data.Close[-1] + self.atr[-1] * 2,
            )
            return

        # MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            if self.position.is_long:
                self.position.close()

            if self.position.is_short or self.is_over_sell():
                return

            self.sell(
                # sl=self.data.Close[-1] * (1 + (self.stop_loss / 100)), # percent
                # tp=self.data.Close[-1] * (1 - (self.take_profit / 100)),
                sl=self.data.Close[-1] + self.atr[-1] * 2,  # atr
                tp=self.data.Close[-1] - self.atr[-1] * 2,  # atr
            )
            return


bt = Backtest(
    yfdata, My_Strategy, cash=10000000, commission=0.003, exclusive_orders=True
)


# 最適化
# optimize = bt.optimize(
#     atr_line=range(120, 350, 10),
#     # take_profit=range(2, 10, 2),
#     # stop_loss=range(2, 10, 2),
#     maximize="Equity Final [$]",
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)


# optimize = bt.optimize(
#     # prop_rsi=range(7, 21, 7),
#     prop_rsi_high=50,
#     prop_rsi_low=30,
#     # take_profit=range(2, 8, 2),
#     # stop_loss=range(2, 8, 2),
#     constraint=lambda p: p.prop_rsi_low < p.prop_rsi_high,
#     maximize="SQN",
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)

# 出力
output = bt.run()
print("Return : " + str(output["Return [%]"]) + "%")
print("Trades : " + str(output["# Trades"]))
print(output)
bt.plot()

4.747490111073871
3.0940951711405207
1.3107216471925476
4.9204673146891365
4.311653518115799
1.7701432731655253
3.7725903218476686
3.630538113434123
2.886517672380018
4.021789512287266
2.3710770285314764
3.51259400539502
3.4375290467396624
0.978758946926761
0.026843101258378077
2.090147220969918
2.170464091061197
2.551636891409914
0.6827062285679641
4.667542628434685
1.9441308526807477
2.3540932895318223
3.6337910389638406
0.09252004555315718
Return : 8.912351169385033%
Trades : 15
Start                     2020-01-02 00:00:00
End                       2020-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                       36.76
Equity Final [$]                  10891235.12
Equity Peak [$]                   11708390.59
Return [%]                               8.91
Buy & Hold Return [%]                   45.27
Return (Ann.) [%]                        8.88
Volatility (Ann.) [%]                   17.78
Sharpe Ratio                             0.50
Sortino

Row(id='33756', ...)

In [7]:
# write HTML file
filename = "Return_" + str(round(output["Return [%]"])) + "%"
bt.plot(filename=filename)


add_html(
    filename + ".html",
    pd.DataFrame(
        [
            {
                "rsi": My_Strategy.prop_rsi,
                "rsi_high": My_Strategy.prop_rsi_high,
                "prop_rsi_low": My_Strategy.prop_rsi_low,
                "memo": "週足RSIと日足MACDを組み合わせた売買方法<br>RSIでMACDのダマシを極力除外する<br>具体的にはRSIがhighとlowの間は購入しない",
            }
        ],
        index=["value"],
    )
    .transpose()
    .to_html(escape=False),
    True,
)
add_html(filename + ".html", pd.DataFrame(output).to_html(), False)

/tmp/ipykernel_111/23335848.py:3: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(filename=filename)
